In [ ]:
! pip -q install gdown

In [ ]:
%cd /content

In [ ]:
import gdown
import zipfile

def download_file_from_google_drive(file_id, output_file):
    """
    Download a file from Google Drive.

    :param file_id: The Google Drive file ID.
    :param output_file: The name of the file to save.
    """
    url = f"https://drive.google.com/uc?id={file_id}"
    gdown.download(url, output_file, quiet=False)

# Example usage:
bone_fracture_detection_file_id = "1--fAnsBtykHxIvq6oqxDAaOJpaWZ5LbP"
bone_out = "bone_fracture_detection.zip"
download_file_from_google_drive(bone_fracture_detection_file_id, bone_out)

# Open the ZIP file for reading
with zipfile.ZipFile(bone_out, 'r') as zip_ref:
    # Extract all the contents to the specified directory
    zip_ref.extractall()

tumor_detection_file_id = "1-2_xDKaO3K1LUtEL-AdGZzZiQP9ulzdM"
tumor_out = "tumor_detection.zip"
download_file_from_google_drive(tumor_detection_file_id, tumor_out)


with zipfile.ZipFile(tumor_out, 'r') as zip_ref:
    # Extract all the contents to the specified directory
    zip_ref.extractall()

In [ ]:
import os
import pathlib

# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models
!pip install numpy --upgrade
# Install the Object Detection API

In [ ]:
%cd /content

In [ ]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

In [ ]:
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from PIL import Image
import numpy as np

In [ ]:
def usemodel(model_path):
  model = tf.saved_model.load(f"{model_path}/saved_model")
  label_map_path = f"{model_path}/labels.pbtxt"
  label_map = label_map_util.load_labelmap(label_map_path)
  categories = label_map_util.convert_label_map_to_categories(label_map,
                                                               max_num_classes=13,
                                                               use_display_name=True)
  return model , categories

def run (model ,categories, image_path , save_name , min_score_thresh=.2 ) :
  category_index = label_map_util.create_category_index(categories)
  image = Image.open(image_path).convert("RGB")
  image_np = np.array(image)
  input_tensor = tf.convert_to_tensor(image_np)
  input_tensor = input_tensor[tf.newaxis, ...]
  output_dict = model(input_tensor)
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key: value[0, :num_detections].numpy() for key, value in output_dict.items()}
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
  viz_utils.visualize_boxes_and_labels_on_image_array(
                                                        image_np,
                                                        output_dict['detection_boxes'],
                                                        output_dict['detection_classes'],
                                                        output_dict['detection_scores'],
                                                        category_index,
                                                        use_normalized_coordinates=True,
                                                        max_boxes_to_draw=200,
                                                        min_score_thresh=min_score_thresh,
                                                        agnostic_mode=False)
  Imageresult = Image.fromarray(image_np)
  Imageresult.save(f"{save_name}.png")
  return Imageresult

In [ ]:
%cd /content

In [ ]:
model_path = "tumor_detection/" # or you can use bone_fracture_detection model

In [ ]:
model , categories = usemodel(model_path)

In [ ]:
run(model ,categories, image_path="Put your Image Path Here" , save_name="result_0" , min_score_thresh=0.3)
# you can change save_name to any name you want , also you can  specify your min_score_thresh (0.1 to 1)